- Application name 	My First API
- API key 	f01b4421cf646ed49cf5fa6e8fb8fad6
- Shared secret 	b8e801a9932d2dccc4ee869e1b613840
- Registered to 	cbueno82
****

- Bio artista
- Popularidad y estadísticas de reproducción
- Artistas similares

In [2]:
import requests
import csv
import pandas as pd
import time

In [3]:
API_KEY = 'f01b4421cf646ed49cf5fa6e8fb8fad6'
BASE_URL = 'http://ws.audioscrobbler.com/2.0/'

PRUEBA: Aquí para ver artistas sueltos sin CSV ni DF. Solo sacamos BIO.

In [22]:
def get_artist_info(artist_name):
    # Parámetros de la consulta
    params = {
        'method': 'artist.getinfo',
        'artist': artist_name,
        'api_key': API_KEY,
        'format': 'json'
    }
    
    response = requests.get(BASE_URL, params=params)
    
    # Comprobar que la petición fue exitosa
    if response.status_code == 200:
        data = response.json()
        if 'error' not in data:
            # Extraer información del artista
            artist_info = data['artist']
            name = artist_info['name']
            bio = artist_info['bio']['summary']
            print(f"Información sobre {name}:\n{bio}")
        else:
            print(f"Error: {data['message']}")
    else:
        print("Error en la solicitud")

# Llamar a la función con el nombre de un artista
get_artist_info('Estopa')

Información sobre Estopa:
Estopa is a pop/flamenco duo founded in 1999 by brothers José and David Muñoz of Cornellà de Llobregat (Baix Llobregat), Catalonia. Their first album, Estopa, has sold well over 1,000,000 copies to date (the equivalent of an American platinum album) and their latest, Voces de Ultrarumba, sold more than 200,000 copies on its first day of release. Their playful, Rumba-tinged compositions have made them one of the most successful pop groups in Spanish history, in terms of both album sales and critical acclaim. <a href="https://www.last.fm/music/Estopa">Read more on Last.fm</a>


Aquí con CSV previo y para sacar un CSV partiendo de un DF. Hay algunos problemillas como que no tengo claro que la información sea exactamente la que necesitamos. 

In [10]:
def get_artist_info(artist_name):
    # Parámetros de la consulta
    params = {
        'method': 'artist.getinfo',
        'artist': artist_name,
        'api_key': API_KEY,
        'format': 'json'
    }
    
    # Hacer la petición GET a la API
    response = requests.get(BASE_URL, params=params)
    
    # Comprobar que la petición fue exitosa
    if response.status_code == 200:
        data = response.json()
        
        if 'error' not in data:
            artist_info = data['artist']
            name = artist_info['name']
            
            # Información de la biografía
            bio_summary = artist_info['bio']['summary']
            
            # Estadísticas de popularidad
            listeners = artist_info['stats']['listeners']
            playcount = artist_info['stats']['playcount']
            
            # Artistas similares
            similar_crudo = artist_info['similar']['artist'][:5]  # limitamos a 5 el número de art. sim.
            similar_artists = tuple(f"{name}-{sim['name']}" for sim in similar_crudo)
            print(type(similar_artists)) # aquí solo estoy asegurando que he guardado una tupla
            
            # Devolver los datos que serán guardados en el CSV
            return {
                'artist_name': name,
                'bio_summary': bio_summary,
                'listeners': listeners,
                'playcount': playcount,
                'similar_artists': ", ".join(similar_artists) 
            }
        else:
            print(f"Error al obtener datos para {artist_name}: {data['message']}")
            return None
    else:
        print(f"Error en la solicitud para {artist_name}")
        return None


def save_to_csv(artists, filename="pop_artistas_lastfm_1.csv"):
    # Crear una lista donde se almacenarán todos los datos de los artistas
    artist_data = []
    
    # Obtener los datos de cada artista y agregarlos a la lista
    for artist in artists:
        artist_info = get_artist_info(artist)
        if artist_info:
            artist_data.append(artist_info)
            print(f"Datos de {artist} guardados correctamente.")
        else:
            print(f"No se pudieron obtener datos para {artist}.")
    
    # Crear el DataFrame a partir de los datos obtenidos
    df = pd.DataFrame(artist_data)
    
    # Guardar el DataFrame como CSV
    df.to_csv(filename, index=False, encoding='utf-8')
    print(f"Datos guardados en {filename}.")
    
    return artist_data  # Ahora devolvemos artist_data para usarlo más tarde

# Lista de artistas para probar uno por uno
artists = ['Estopa', 'Danza Invisible', 'La Oreja de VanGogh', 'Presuntos Implicados']

# Guardar la información en un CSV y obtener los datos de los artistas
artist_data = save_to_csv(artists)

# Crear el DataFrame a partir de los datos obtenidos
df = pd.DataFrame(artist_data)

# Mostrar el DataFrame
print(df.head(2))  # Muestra las primeras filas del DataFrame

<class 'tuple'>
Datos de Estopa guardados correctamente.
<class 'tuple'>
Datos de Danza Invisible guardados correctamente.
<class 'tuple'>
Datos de La Oreja de VanGogh guardados correctamente.
<class 'tuple'>
Datos de Presuntos Implicados guardados correctamente.
Datos guardados en pop_artistas_lastfm_1.csv.
       artist_name                                        bio_summary  \
0           Estopa  Estopa is a pop/flamenco duo founded in 1999 b...   
1  Danza Invisible  A band from Torremolinos, Málaga, very popular...   

  listeners playcount                                    similar_artists  
0    283396   7373795  Estopa-Melendi, Estopa-El Canto del Loco, Esto...  
1     58544    434573  Danza Invisible-Radio Futura, Danza Invisible-...  


In [17]:
def get_artist_info(artist_name):
    params = {
        'method': 'artist.getinfo',
        'artist': artist_name,
        'api_key': API_KEY,
        'format': 'json'
    }

    try:
        response = requests.get(BASE_URL, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()

        if 'artist' not in data:
            print(f"No se encontró información para {artist_name}")
            return

        artist_info = data['artist']
        name = artist_info['name']
        bio = artist_info['bio']['summary']
        stats = artist_info['stats']
        listeners = stats['listeners']
        playcount = stats['playcount']

        similar_crudo = artist_info['similar']['artist'][:5]
        similar_artists = tuple(f"{name}-{sim['name']}" for sim in similar_crudo)

        print(f"{name}")
        print(f"Oyentes: {listeners}")
        print(f"Reproducciones: {playcount}")
        print(f"Bio: {bio}")  
        print(f"Similares: {similar_artists}")
        print(" ")

        return {
            'name': name,
            'listeners': listeners,
            'playcount': playcount,
            'bio': bio,
            'similar_artists': ", ".join(similar_artists)
        }

    except requests.RequestException as e:
        print(f"Error de red al obtener datos para {artist_name}: {e}")
    except (KeyError, ValueError) as e:
        print(f"Error al procesar datos para {artist_name}: {e}")


def read_artists_from_csv(filename):
    artists = []
    with open(filename, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            if len(row) >= 2:       # aseguramos que hay al menos 2 columnas
                artists.append(row[1].strip())  # tomamos la segunda columna (índice 1)
    return artists

def write_results_to_csv(results, output_filename):
    with open(output_filename, mode='w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['name', 'listeners', 'playcount', 'bio', 'similar_artists']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in results:
            writer.writerow(row)

In [ ]:
# NO PONGÁIS ESTO A FUNCIONAR A LO LOCO, EN LA SIGUIENTE CELDA HAY UNA MINIPRUEBA, ESTO TE SACA UN CHORRO DE INFO 

# Ejecución
results = []
for artist in read_artists_from_csv('pop_artistas.csv'):
    info = get_artist_info(artist)
    if info:
        results.append(info)
        
write_results_to_csv(results, 'pop_artistas_lastfm.csv')


In [18]:
def prueba_csv(filename):
    artists = []
    with open(filename, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            if len(row) > 1:          
                artists.append(row[1])  
    return artists

results = []

artists = prueba_csv('pop_artistas.csv')

for artist in artists[:5]: 
    info = get_artist_info(artist)
    print(f"{artist} {info}")
    if info:
        results.append(info)

0
Oyentes: 8479
Reproducciones: 42601
Bio: 1. 0 is a black metal band from Iceland, consisting of:

D.G. - Guitars
TMS - Drums
S.S. - Vocals
Ö.S. - Guitars
H.K.F. - Bass 

They have released two full length albums to date; 2014's Null & Void, and 2020's Entity. <a href="https://www.last.fm/music/+noredirect/0">Read more on Last.fm</a>
Similares: ()
 
0 {'name': '0', 'listeners': '8479', 'playcount': '42601', 'bio': '1. 0 is a black metal band from Iceland, consisting of:\n\nD.G. - Guitars\nTMS - Drums\nS.S. - Vocals\nÖ.S. - Guitars\nH.K.F. - Bass \n\nThey have released two full length albums to date; 2014\'s Null & Void, and 2020\'s Entity. <a href="https://www.last.fm/music/+noredirect/0">Read more on Last.fm</a>', 'similar_artists': ''}
Neal Hefti
Oyentes: 23074
Reproducciones: 122162
Bio: Neal Hefti (born October 29, 1922 in Hastings, Nebraska - died October 11, 2008 in Toluca Lake, California) was an American jazz trumpeter, composer, and arranger.

He began arranging professionall